In [32]:
from utilities import *
from core.initial import Insertions
from core.solver import CGCPSolver
from docplex.cp.model import *
from docplex.mp.model import Model
import pandas as pd

In [2]:
datafile = "D:/Google Drive/MSc MST-AUEB/_Thesis_/Main Thesis/Model Data.xlsx"
d = DataProvider(filepath=datafile, route='910')
model = CSPModel(d)
model.build_model()
initial = Insertions(model)
initial.solve()
s = CGCPSolver(initial.sol)

In [ ]:
rmp = Model(name='Restricted_Master_Problem')

rmp.trips = s.trips
rmp.duties = s.duties

rmp.trip_duty_arr = rmp.binary_var_matrix(s.trips, s.duties)

In [ ]:
rmp.duty_arr = rmp.binary_var_list(rmp.duties, name='duty')

In [ ]:
for trip in rmp.trips:
    duty_ct = rmp.sum(rmp.trip_duty_arr[(trip, duty)] * rmp.duty_arr[duty.ID] for duty in rmp.duties) == 1
    rmp.add_constraint(duty_ct)

In [ ]:
rmp.minimize(rmp.sum(rmp.duty_arr[duty.ID] for duty in rmp.duties))

In [ ]:
rmp.solve()

In [34]:
sub = CpoModel(name="Pricing_Subproblem")

In [35]:
trips = [interval_var(start=trip.start_time, end=trip.end_time, length=trip.duration, size=trip.duration, name=f'Trip_{idx}') for idx, trip in enumerate(model.trips)]

In [53]:
ntrips = len(model.trips)

duties = [interval_var(length=model.constraints.shift_span, size=model.constraints.shift_span, name=f"Duty_{i}") for i in range(ntrips)]

In [46]:
td_matrix = {}
for trip in model.trips:
    for duty in range(ntrips):
        td_matrix[(trip.ID, duty)] = binary_var(name=f"T{trip.ID}-D{duty}")

{(0, 0): <docplex.cp.expression.CpoIntVar at 0x1965fb9cdb0>,
 (0, 1): <docplex.cp.expression.CpoIntVar at 0x1965fba3680>,
 (0, 2): <docplex.cp.expression.CpoIntVar at 0x1965fba3720>,
 (0, 3): <docplex.cp.expression.CpoIntVar at 0x1965fba3770>,
 (0, 4): <docplex.cp.expression.CpoIntVar at 0x1965fba37c0>,
 (0, 5): <docplex.cp.expression.CpoIntVar at 0x1965fba3810>,
 (0, 6): <docplex.cp.expression.CpoIntVar at 0x1965fba3860>,
 (0, 7): <docplex.cp.expression.CpoIntVar at 0x1965fba38b0>,
 (0, 8): <docplex.cp.expression.CpoIntVar at 0x1965fba3900>,
 (0, 9): <docplex.cp.expression.CpoIntVar at 0x1965fba3950>,
 (0, 10): <docplex.cp.expression.CpoIntVar at 0x1965fba39a0>,
 (0, 11): <docplex.cp.expression.CpoIntVar at 0x1965fba39f0>,
 (0, 12): <docplex.cp.expression.CpoIntVar at 0x1965fba3a40>,
 (0, 13): <docplex.cp.expression.CpoIntVar at 0x1965fba3a90>,
 (0, 14): <docplex.cp.expression.CpoIntVar at 0x1965fba3ae0>,
 (0, 15): <docplex.cp.expression.CpoIntVar at 0x1965fba3b30>,
 (0, 16): <docplex

In [44]:
duties[0].end

(0, 4503599627370494)

In [33]:
INTERVAL_MAX

4503599627370494